In [ ]:
from sys import exit
import os
import json
from glob import glob

import numpy as np

from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Image2Pipeline
from jwst.pipeline import calwebb_image2

from astropy.table import Table
from astropy.io import fits


# Excuting the JWST pipeline on the  JWST astrometric calibration field in the LMC (part 1)

This Notebook must be executed from an environment that has the JWST pipeline installed. In this tutorial we show step by step instructions on how to run the JWST pipeline on the simulated images created using the previous notebook.

This notebook executes the first two steps of the NIRCam pipeline on the images that were created with the simulator MIRAGE in `preimaging_01_mirage`. The example shows how to execute the pipeline on the F150W images only. 

## Setting things up

In [ ]:
path='./.' # this is the working directory
os.chdir(path)
print(os.getcwd())

In [ ]:
filtname = 'f150w'

In [ ]:
# gather the F150W yaml files back in the working directory
cwd = os.getcwd()
filter_pattern = os.path.join(cwd, 'yaml', filtname.lower(), 'jw*yaml') 
files = glob(filter_pattern)[:]
inputyaml = files
namelist = []
outlist = []

We define the calwebb_detector1 configuration file to use in the pipeline call.

In [ ]:
calwebb_detector1_file = os.path.join('preimaging', 'calwebb_detector1.cfg')

## Executing calwebb_detector1

We select the *_uncal.fits files and execute calwebb_detector1 to generate the *_rate.fits files.

In [ ]:
for input_yaml_path in inputyaml:
    print(input_yaml_path) 
    
    base = os.path.basename(input_yaml_path)
    name = os.path.splitext(base)[0]

    exi = os.path.exists(name + '_rate.fits')
    if exi:
        continue
        
    namelist.append(name + '_uncal.fits')
    outlist.append(name + '.fits')
    
print(namelist)
print(outlist)

files  = namelist 
outfile = outlist 


for k, filename in enumerate(files):
    Detector1Pipeline.call(filename, config_file=calwebb_detector1_file, 
                           output_file=outfile[k]);


Here we run the second stage of the pipeline.

In [ ]:
# gather the F150W yaml files back in the working directory
cwd = os.getcwd()
filter_pattern = os.path.join(cwd, 'yaml', filtname.lower(), 'jw*yaml') 
files = glob(filter_pattern)[:] # Use [0:2] to test the first two files.
inputyaml = files
namelist = []
outlist = []

## Executing calwebb_image2

We select the *_rate.fits files and execute calwebb_image2 to generate the *_cal.fits files.

In [ ]:
for input_yaml_path in inputyaml:
    print(input_yaml_path)
    base = os.path.basename(input_yaml_path)
    name = os.path.splitext(base)[0]
   
    exi = os.path.exists(name + '_cal.fits')
    if exi:
        continue

    namelist.append(name + '_rate.fits')
    outlist.append(name + '.fits')
        
print(namelist)
    
files = namelist 
outfile = outlist 

Generate the _cal.fits files.

In [ ]:
for k, filename in enumerate(files):
    print('-'*60)
    print(filename)

    m  =  calwebb_image2.Image2Pipeline(output_file=outfile[k])
    m.run(filename)
   